In [1]:
import pandas as pd
from statsbombpy import sb
import streamlit as st
import warnings
warnings.filterwarnings('ignore')


In [2]:
def get_data(match_ids):
    event_data_tot=pd.DataFrame()
    for match_id in match_ids:
        event_data=sb.events(match_id=match_id)
        try:
            df_360=pd.read_json(f"/Users/borgwardt/Documents/repos/open-data/data/three-sixty/{match_id}.json")
        except:
            print(match_id)
        df_merged=pd.merge(event_data,df_360,how="left",left_on="id",right_on="event_uuid")
        event_data_tot=pd.concat([event_data_tot, df_merged], ignore_index=True)
    return event_data_tot
def preprocess_data(df_raw):
    df_preprocessed=df_raw.sort_values(["match_id","minute","timestamp"])
    df_preprocessed.reset_index(inplace=True)
    return df_preprocessed

def create_kpis(team_events):
    # Total shots
    shots = len(team_events[team_events["type"] == "Shot"])
    # Total xg
    shot_statsbomb_xg=team_events["shot_statsbomb_xg"].sum()
    # Total passes
    passes = len(team_events[team_events["type"] == "Pass"])

    # Pass accuracy
    completed_passes = len(team_events[(team_events["type"] == "Pass") & (team_events["pass_outcome"].isnull())])
    pass_accuracy = (completed_passes / passes) * 100

    # Total duels won
    duels_won = len(team_events[(team_events["type"] == "Duel") & (team_events["duel_outcome"] == "Won")])

    # Total tackles
    tackles = len(team_events[team_events["type"] == "Tackle"])

    # Total interceptions
    interceptions = len(team_events[team_events["type"] == "Interception"])

    # Total clearances
    clearances = len(team_events[team_events["type"] == "Clearance"])

    # Percentage of possession 
    team_possession_seconds=team_events[(team_events['type']!="Pressure")].duration.sum()
    kpi_summary=pd.DataFrame({
        "shots": [shots],
        'shot_statsbomb_xg':[shot_statsbomb_xg],
        "passes": [passes],
        "pass_accuracy": [pass_accuracy],
        "duels_won": [duels_won],
        "tackles": [tackles],
        "interceptions": [interceptions],
        "clearances": [clearances],
        "possession_seconds": [team_possession_seconds]
    })
    print(kpi_summary)
    return kpi_summary

In [3]:
competitions = sb.competitions()
womens_euro_competition = competitions[competitions['competition_name'] == "UEFA Women's Euro"]
womens_euro_2022 = womens_euro_competition[womens_euro_competition['season_name'] == '2022']
euro_competition_id=womens_euro_2022.competition_id.unique()[0]
euro_season_id=womens_euro_2022.season_id.unique()[0]
match_ids=sb.matches(competition_id=euro_competition_id, season_id=euro_season_id).match_id

In [4]:
df_raw=get_data(match_ids)

In [5]:
df_preprocessed=preprocess_data(df_raw)
df_preprocessed
df_kpis=df_preprocessed.groupby(["match_id","team"]).apply(create_kpis)
df_kpis

ValueError: If using all scalar values, you must pass an index

In [6]:
pd.DataFrame({
        "shots": 1,
        'shot_statsbomb_xg':1,
        "passes": 1,
        "pass_accuracy": 1,
        "duels_won": 1,
        "tackles": 1,
        "interceptions": 1,
        "clearances": 1,
        "possession_seconds": 1
    })

ValueError: If using all scalar values, you must pass an index

In [ ]:
pd.read_json("/Users/borgwardt/Documents/repos/open-data/data/three-sixty/3835342.json")

,event_uuid,visible_area,freeze_frame
0,fde0e241-0ff3-4faf-9806-bbcd9d0bfb28,"[39.6248103518052, 77.602435306958, 0.0, 6.433...","[{'teammate': True, 'actor': False, 'keeper': ..."
1,fa701824-c7c5-4597-8b11-68ec53e0f68e,"[37.0973922997033, 75.3607031690491, 0.0, 18.6...","[{'teammate': True, 'actor': False, 'keeper': ..."
2,6f1761f2-f827-4c57-a960-b64b14b407f9,"[37.0973922997033, 75.3607031690491, 0.0, 18.6...","[{'teammate': True, 'actor': False, 'keeper': ..."
3,59c03233-bf02-407e-8c62-39ab4c9d8795,"[13.699331198296, 80.0, 45.5966437365325, 8.34...","[{'teammate': True, 'actor': False, 'keeper': ..."
4,de12a14f-f7c9-407a-9250-49ec6689fba8,"[32.8918846791763, 76.423516510925, 0.0, 28.94...","[{'teammate': True, 'actor': False, 'keeper': ..."
...,...,...,...
2797,be94d3f3-80af-460b-a4ac-90d8bddb0882,"[107.715623164502, 80.0, 82.4999149834406, 27....","[{'teammate': False, 'actor': False, 'keeper':..."
2798,58f3c71a-60bf-4354-8e58-b7fd73590604,"[15.9252544426531, 60.9847502771868, 0.0, 45.9...","[{'teammate': True, 'actor': False, 'keeper': ..."
2799,7aeb57d4-6433-4772-9e90-fc9138e58f40,"[15.9252544426531, 60.9847502771868, 0.0, 45.9...","[{'teammate': False, 'actor': False, 'keeper':..."
2800,7399d8d7-a0a1-4a2d-b256-c17acab1efc0,"[113.821253405624, 80.0, 84.7732651830841, 25....","[{'teammate': True, 'actor': False, 'keeper': ..."
